In [1]:
using Pkg

In [2]:
Pkg.activate("../")

  Activating project at `~/haxx/RNAForecasterPaperCode`


In [3]:
include("../trainRNAForecaster.jl")

createEnsembleForecaster

In [4]:
include("../makeRecursivePredictions.jl")

mostTimeVariableGenes

In [11]:
# Let's load a trained model
using Flux: loadmodel!
using JLD2
function loadForecaster(fileName::String, inputNodes::Int, hiddenLayerNodes::Int)
    #recreate neural network structure
    nn = Chain(Dense(inputNodes, hiddenLayerNodes, relu),
               Dense(hiddenLayerNodes, inputNodes))
    model = NeuralODE(nn, (0.0f0, 1.0f0), Tsit5(),
                       save_everystep = false,
                       reltol = 1e-3, abstol = 1e-3,
                       save_start = false)
    #load parameters into the model
    model = loadmodel!(model, load_object(fileName))
    return model
end



NeuralODE()

In [12]:
trained = loadForecaster("pancNeuralODEResult.jld2", 1352, 6000)

In [42]:
target = Float32.(splicedSub[:,[1701,2601]])
size(target)

(1352, 2)

In [43]:
futures = predictCellFutures(trained,target,50)

1352×2×50 Array{Float32, 3}:
[:, :, 1] =
 0.0         0.0
 0.0         0.0
 4.0         4.0
 2.0         2.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.00817373  0.745167
 0.0         0.0
 ⋮           
 2.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 2.0         2.0

[:, :, 2] =
 0.0       0.0
 0.892818  0.0
 4.0       4.0
 2.0       2.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.364751  0.94548
 0.0       0.0
 ⋮         
 2.0       0.365104
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 2.0       2.0

[:, :, 3] =
 0.0       0.0
 1.30637   0.0
 4.0       4.0
 2.0       2.0
 0.0       0.0
 0.0       0.0
 0.

In [46]:
writedlm("./trajectories_2600.tsv",futures[:,1,:])
writedlm("./trajectories_1700.tsv",futures[:,2,:])

In [16]:
using DelimitedFiles
spliced = readdlm("pancSplicedCountsHVGs.csv", ',')
unspliced = readdlm("pancUnsplicedCountsHVGs.csv", ',')

zeroPropSplicedGenes = Array{Float64}(undef, size(spliced)[1])
for i=1:size(spliced)[1]
    zeroPropSplicedGenes[i] = length(findall(x->x ==0, spliced[i,:]))/size(spliced)[2]
end
zeroPropUnsplicedGenes = Array{Float64}(undef, size(unspliced)[1])
for i=1:size(unspliced)[1]
    zeroPropUnsplicedGenes[i] = length(findall(x->x ==0, unspliced[i,:]))/size(unspliced)[2]
end

splicedSub = spliced[intersect(findall(x->x < 0.98, zeroPropSplicedGenes), findall(x->x < 0.98, zeroPropUnsplicedGenes)),:]
unsplicedSub = unspliced[intersect(findall(x->x < 0.98, zeroPropSplicedGenes), findall(x->x < 0.98, zeroPropUnsplicedGenes)),:]

geneNames = readdlm("pancHVGNames.csv", ',')
geneNames = geneNames[intersect(findall(x->x < 0.98, zeroPropSplicedGenes), findall(x->x < 0.98, zeroPropUnsplicedGenes))]
geneNames = string.(geneNames)


1352-element Vector{String}:
 "Cops5"
 "Sbspon"
 "Pkhd1"
 "Fam135a"
 "Zfp451"
 "Tbc1d8"
 "Fhl2"
 "Pgap1"
 "Ankrd44"
 "Tyw5"
 "Raph1"
 "Klf7"
 "Akr1cl"
 ⋮
 "Mum1l1"
 "Pak3"
 "Alg13"
 "Wnk3"
 "Sh3kbp1"
 "Map3k15"
 "Rai2"
 "Ap1s2"
 "Tmem27"
 "Ddx3y"
 "Eif2s3y"
 "Erdr1"

In [29]:
writedlm("ZPSGenes.tsv",geneNames)